In [ ]:
import pandas as pd
import numpy as np
import re
import gensim
#import contractions
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import gensim
from gensim.utils import simple_preprocess
from sklearn.cluster import KMeans
from gensim.models import Word2Vec
from sklearn import cluster
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from nltk.cluster import KMeansClusterer, euclidean_distance
import nltk
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
import seaborn as sns
from wordcloud import WordCloud
from nltk.tokenize import wordpunct_tokenize
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
#import flair
from scipy.spatial import distance
#from simpletransformers.language_representation import RepresentationModel
import requests
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
#import umap.umap_ as umap
from sklearn.decomposition import PCA
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import silhouette_score
from wordcloud import WordCloud
#import texthero as hero
from sklearn import metrics
%matplotlib inline

In [ ]:
!pip install -U sentence-transformers
#!pip install "gensim==3.8.1"
#!pip install texthero
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 2.4 MB/s 
     |████████████████████████████████| 4.2 MB 14.2 MB/s 
     |████████████████████████████████| 1.2 MB 38.8 MB/s 
     |████████████████████████████████| 86 kB 5.9 MB/s 
     |████████████████████████████████| 596 kB 48.6 MB/s 
     |████████████████████████████████| 6.6 MB 49.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=af227153fd5bf70f0f1785f2620c0efa6bb42c0ba6af0f16da45d29a4d197fe6
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wh

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_train=pd.read_csv('/content/drive/MyDrive/BBC News Train.csv')

In [ ]:
data_train.drop(['ArticleId'],inplace=True,axis=1)

In [ ]:
vec = TfidfVectorizer()
features= vec.fit_transform(data_train['Text'])
print("Input features shape:", features.shape)
print(f"\nTake a look at the features extracted from the first news article:\n{features[0].toarray()}")

Input features shape: (1490, 24746)

Take a look at the features extracted from the first news article:
[[0. 0. 0. ... 0. 0. 0.]]


In [ ]:
!pip install fuzzy-c-means

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.1 MB 36.3 MB/s 


In [ ]:
from fcmeans import FCM
features=features.toarray()

In [ ]:
my_model = FCM(n_clusters=5,random_state=42) 
my_model.fit(features)
data_train['cluster']=my_model.predict(features)

In [ ]:
cluster_to_category = {}
for cat in data_train['Category'].unique():
    mark = data_train['Category'] == cat
    top = data_train[mark]['cluster'].value_counts().head(1)
    count = top.values[0]
    cluster = top.index[0]
    print(f"{cat}:\n Top cluster number: {cluster}, Number of samples: {count}")
    cluster_to_category[cluster] = cat

print("\nMap cluster number to category:")
cluster_to_category

business:
 Top cluster number: 2, Number of samples: 168
tech:
 Top cluster number: 4, Number of samples: 92
politics:
 Top cluster number: 1, Number of samples: 240
sport:
 Top cluster number: 2, Number of samples: 298
entertainment:
 Top cluster number: 2, Number of samples: 237

Map cluster number to category:


{1: 'politics', 2: 'entertainment', 4: 'tech'}

In [ ]:
data_train['clustered_category'] = data_train['cluster'].map(cluster_to_category)
data_train.head(20)

,Text,Category,cluster,clustered_category
0,worldcom ex-boss launches defence lawyers defe...,business,2,entertainment
1,german business confidence slides german busin...,business,3,NaN
2,bbc poll indicates economic gloom citizens in ...,business,4,tech
3,lifestyle governs mobile choice faster bett...,tech,4,tech
4,enron bosses in $168m payout eighteen former e...,business,0,NaN
5,howard truanted to play snooker conservative...,politics,2,entertainment
6,wales silent on grand slam talk rhys williams ...,sport,2,entertainment
7,french honour for director parker british film...,entertainment,2,entertainment
8,car giant hit by mercedes slump a slump in pro...,business,2,entertainment
9,fockers fuel festive film chart comedy meet th...,entertainment,2,entertainment


In [ ]:
print('Overall accuracy of clustered categories:', np.mean(data_train['Category'] == data_train['clustered_category']))

Overall accuracy of clustered categories: 0.3818791946308725


In [ ]:
from sklearn.metrics import silhouette_score
print("Silhouette Coefficient: %0.3f"% metrics.silhouette_score(features,data_train['cluster']))

Silhouette Coefficient: -0.004


#Tf-Idf Preprocessed Data +C-means


In [ ]:
import pandas as pd
import numpy as np
import re
import gensim
import contractions
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import gensim
from gensim.utils import simple_preprocess
from sklearn.cluster import KMeans
from gensim.models import Word2Vec
from sklearn import cluster
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from nltk.cluster import KMeansClusterer, euclidean_distance
import nltk
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
import seaborn as sns
from wordcloud import WordCloud
from nltk.tokenize import wordpunct_tokenize
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
nltk.download('stopwords') 
nltk.download('wordnet')
nltk.download('omw-1.4')
data_train=pd.read_csv('/content/drive/MyDrive/BBC News Train.csv')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


In [ ]:
STOPWORDS = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
data_train.drop(['ArticleId'],axis=1,inplace=True)

In [ ]:
def expand_contractions(text):
    expanded_words = []    
    for word in text.split():
        expanded_words.append(contractions.fix(word))   
    expanded_text = ' '.join(expanded_words)
    return expanded_text
 
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
def remove_digits(text):
    return " ".join([word for word in str(text).split() if not(word.isdigit())])
def remove_shorttokens(text):
    return " ".join([word for word in str(text).split() if len(word)>1])
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])
#convert it to string
data_train["Text"]=data_train["Text"].astype(str)
#Convert to lower and strip
data_train["Text"]=data_train["Text"].str.lower().str.strip()
#apply contractions
data_train["Text"]=data_train["Text"].apply(lambda x:expand_contractions(x))
#remove punctuations
data_train["Text"]=data_train["Text"].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '' , x))
#remove stop words
data_train["Text"]=data_train["Text"].apply(lambda x: remove_stopwords(x))
#Lemmatize the sentence
data_train["Text"]=data_train["Text"].apply(lambda text: lemmatize_words(text))
data_train["Text"]=data_train["Text"].apply(lambda text:remove_digits(text))
data_train["Text"]=data_train["Text"].apply(lambda text:remove_shorttokens(text))

In [ ]:
vec = TfidfVectorizer()
features= vec.fit_transform(data_train['Text'])
print("Input features shape:", features.shape)
print(f"\nTake a look at the features extracted from the first news article:\n{features[0].toarray()}")
features=features.toarray()

Input features shape: (1490, 23815)

Take a look at the features extracted from the first news article:
[[0. 0. 0. ... 0. 0. 0.]]


In [ ]:
my_model = FCM(n_clusters=5,random_state=42) 
my_model.fit(features)
data_train['cluster']=my_model.predict(features)

In [ ]:
cluster_to_category = {}
for cat in data_train['Category'].unique():
    mark = data_train['Category'] == cat
    top = data_train[mark]['cluster'].value_counts().head(1)
    count = top.values[0]
    cluster = top.index[0]
    print(f"{cat}:\n Top cluster number: {cluster}, Number of samples: {count}")
    cluster_to_category[cluster] = cat

print("\nMap cluster number to category:")
cluster_to_category

business:
 Top cluster number: 4, Number of samples: 179
tech:
 Top cluster number: 2, Number of samples: 179
politics:
 Top cluster number: 1, Number of samples: 228
sport:
 Top cluster number: 2, Number of samples: 345
entertainment:
 Top cluster number: 2, Number of samples: 261

Map cluster number to category:


{1: 'politics', 2: 'entertainment', 4: 'business'}

In [ ]:
data_train['clustered_category'] = data_train['cluster'].map(cluster_to_category)
print('Overall accuracy of clustered categories:', np.mean(data_train['Category'] == data_train['clustered_category']))

Overall accuracy of clustered categories: 0.4483221476510067


In [ ]:
from sklearn.metrics import silhouette_score
print("Silhouette Coefficient: %0.3f"% metrics.silhouette_score(features,data_train['cluster']))

Silhouette Coefficient: 0.004


#Experiment with PCA

In [ ]:
data_embbedding=features
x=[]
y=[]
for i in range(1490,0,-50):
  pca_2 = PCA(n_components=i)
  pca_2_result = pca_2.fit_transform(data_embbedding)
  x.append(i)
  y.append(round(np.sum(pca_2.explained_variance_ratio_),2))
  print('Cumulative variance explained by {} principal components: {:.2%}'.format(i,np.sum(pca_2.explained_variance_ratio_)))

Cumulative variance explained by 1490 principal components: 100.00%
Cumulative variance explained by 1440 principal components: 100.00%
Cumulative variance explained by 1390 principal components: 99.82%
Cumulative variance explained by 1340 principal components: 99.07%
Cumulative variance explained by 1290 principal components: 98.10%
Cumulative variance explained by 1240 principal components: 97.00%
Cumulative variance explained by 1190 principal components: 95.53%
Cumulative variance explained by 1140 principal components: 94.11%
Cumulative variance explained by 1090 principal components: 92.57%
Cumulative variance explained by 1040 principal components: 90.91%
Cumulative variance explained by 990 principal components: 89.11%
Cumulative variance explained by 940 principal components: 87.19%
Cumulative variance explained by 890 principal components: 85.15%
Cumulative variance explained by 840 principal components: 82.96%
Cumulative variance explained by 790 principal components: 80.64

In [ ]:
#95%,90%,85% covaience-->1160,1020,885
vec_95 = TfidfVectorizer(
        max_df=0.5,
        min_df=2,
        stop_words="english",
        max_features=1160,
        use_idf=True
    )
vec_90 = TfidfVectorizer(
        max_df=0.5,
        min_df=2,
        stop_words="english",
        max_features=1020,
        use_idf=True
    )
vec_85 = TfidfVectorizer(
        max_df=0.5,
        min_df=2,
        stop_words="english",
        max_features=885,
        use_idf=True
    )
features95= vec_95.fit_transform(data_train['Text'])
features90=vec_90.fit_transform(data_train['Text'])
features85=vec_85.fit_transform(data_train['Text'])

In [ ]:
features95=features95.toarray()
features90=features90.toarray()
features85=features85.toarray()

In [ ]:
my_model95= FCM(n_clusters=5,random_state=42) 
my_model95.fit(features95)
data_train['cluster95']=my_model95.predict(features95)

my_model90= FCM(n_clusters=5,random_state=42) 
my_model90.fit(features90)
data_train['cluster90']=my_model90.predict(features90)

my_model85= FCM(n_clusters=5,random_state=42) 
my_model85.fit(features85)
data_train['cluster85']=my_model85.predict(features85)

In [ ]:
cluster_to_category95= {}
for cat in data_train['Category'].unique():
    mark = data_train['Category'] == cat
    top = data_train[mark]['cluster95'].value_counts().head(1)
    count = top.values[0]
    cluster = top.index[0]
    print(f"{cat}:\n Top cluster number: {cluster}, Number of samples: {count}")
    cluster_to_category95[cluster] = cat

print("\nMap cluster number to category:")
cluster_to_category95

business:
 Top cluster number: 4, Number of samples: 317
tech:
 Top cluster number: 4, Number of samples: 150
politics:
 Top cluster number: 1, Number of samples: 236
sport:
 Top cluster number: 2, Number of samples: 345
entertainment:
 Top cluster number: 2, Number of samples: 252

Map cluster number to category:


{1: 'politics', 2: 'entertainment', 4: 'tech'}

In [ ]:
data_train['clustered_category95'] = data_train['cluster'].map(cluster_to_category95)
print('Overall accuracy of clustered categories:', np.mean(data_train['Category'] == data_train['clustered_category95']))

Overall accuracy of clustered categories: 0.37919463087248323


In [ ]:
from sklearn.metrics import silhouette_score
print("Silhouette Coefficient: %0.3f"% metrics.silhouette_score(features95,data_train['cluster95']))

Silhouette Coefficient: 0.010


In [ ]:
cluster_to_category90= {}
for cat in data_train['Category'].unique():
    mark = data_train['Category'] == cat
    top = data_train[mark]['cluster90'].value_counts().head(1)
    count = top.values[0]
    cluster = top.index[0]
    print(f"{cat}:\n Top cluster number: {cluster}, Number of samples: {count}")
    cluster_to_category90[cluster] = cat

print("\nMap cluster number to category:")
cluster_to_category90

business:
 Top cluster number: 4, Number of samples: 319
tech:
 Top cluster number: 4, Number of samples: 149
politics:
 Top cluster number: 1, Number of samples: 237
sport:
 Top cluster number: 2, Number of samples: 344
entertainment:
 Top cluster number: 2, Number of samples: 252

Map cluster number to category:


{1: 'politics', 2: 'entertainment', 4: 'tech'}

In [ ]:
data_train['clustered_category90'] = data_train['cluster'].map(cluster_to_category90)
print('Overall accuracy of clustered categories:', np.mean(data_train['Category'] == data_train['clustered_category90']))

Overall accuracy of clustered categories: 0.37919463087248323


In [ ]:
from sklearn.metrics import silhouette_score
print("Silhouette Coefficient: %0.3f"% metrics.silhouette_score(features90,data_train['cluster90']))

Silhouette Coefficient: 0.009


In [ ]:
cluster_to_category85= {}
for cat in data_train['Category'].unique():
    mark = data_train['Category'] == cat
    top = data_train[mark]['cluster85'].value_counts().head(1)
    count = top.values[0]
    cluster = top.index[0]
    print(f"{cat}:\n Top cluster number: {cluster}, Number of samples: {count}")
    cluster_to_category85[cluster] = cat

print("\nMap cluster number to category:")
cluster_to_category85

business:
 Top cluster number: 4, Number of samples: 317
tech:
 Top cluster number: 4, Number of samples: 149
politics:
 Top cluster number: 1, Number of samples: 239
sport:
 Top cluster number: 2, Number of samples: 344
entertainment:
 Top cluster number: 2, Number of samples: 250

Map cluster number to category:


{1: 'politics', 2: 'entertainment', 4: 'tech'}

In [ ]:
data_train['clustered_category85'] = data_train['cluster'].map(cluster_to_category85)
print('Overall accuracy of clustered categories:', np.mean(data_train['Category'] == data_train['clustered_category85']))

Overall accuracy of clustered categories: 0.37919463087248323


In [ ]:
from sklearn.metrics import silhouette_score
print("Silhouette Coefficient: %0.3f"% metrics.silhouette_score(features85,data_train['cluster85']))

Silhouette Coefficient: 0.010
